In [ ]:
#|default_exp text.huggingface

In [ ]:
#|exporti
# Contains code from:
# blurr - Apache License 2.0 - Copyright (c) Wayde Gilliam

In [ ]:
#|export
from __future__ import annotations

import inspect, warnings
from typing import Dict, Iterable, Sequence

import torch._dynamo as dynamo
from torch.utils.data import Sampler, Dataset
from torch.utils.data import DataLoader as _DataLoader
from torch.utils.data.dataloader import _worker_init_fn_t, _collate_fn_t

from transformers import PreTrainedModel
from transformers.modeling_utils import ModuleUtilsMixin
from transformers import logging as hf_logging

from fastai.callback.core import Callback
from fastai.losses import BaseLoss

from fastxtend.imports import *

# Hugging Face Transformers Compatibility
> Train Hugging Face Transformers models using fastai

fastxtend provides basic compatibility for training Hugging Face [Transformers](https://huggingface.co/docs/transformers/index) models using the `fastai.learner.Learner`.

:::{.callout-tip collapse="true"}
#### Tip: Use blurr For a Complete Transformers Integration

[blurr](https://ohmeow.github.io/blurr) provides a complete Hugging Face Transformers integration with fastai, including working fastai datablocks, dataloaders, and other fastai methods.

In contrast, fastxtend only provides basic `Learner` compatibility.
:::

:::{.callout-important collapse="true"}
#### Important: Requires PyTorch 2.0+

fastxtend's Transformers compatibility requires a minimum of PyTorch 2.0.
:::

To use fastxend's compatibility, setup the Hugging Face dataset, dataloader, and model per the [Transformers documentation](https://huggingface.co/docs/transformers/index), exchanging the PyTorch `Dataloader` for the `HuggingFaceLoader`. Then wrap the dataloaders in `fastai.data.core.DataLoaders` and create a `Learner` with the Hugging Face model, `HuggingFaceLoss`, and `HuggingFaceCallback`. This will automatically setup the compatibility and use the Hugging Face model's built in loss.

Jump to the [example](#example) section for a full tutorial.

```python
from fastai.text.all import *
from fastxtend.text.all import *

# load a task specific AutoModel
hf_model = AutoModel.from_pretrained("model-name")

# setup dataset and then dataloaders
train_dataset = dataset['train'].with_format('torch')
train_dataloader = HuggingFaceLoader(
    train_dataset, batch_size=batch_size,
    collate_fn=data_collator, shuffle=True,
    drop_last=True, num_workers=num_cpus()
)

# valid_dataloader definition cut for brevity
dls = DataLoaders(train_dataloader, valid_dataloader)

learn = Learner(dls, hf_model, loss_func=HuggingFaceLoss(), ...,
                cbs=HuggingFaceCallback()).to_bf16()

# save the model after training using Transformers
learn.hf_model.save_pretrained("trained-model-name")
```

To train with a different loss, pass in a PyTorch compatible loss to `Learner` as normal, and `HuggingFaceCallback` will use it instead of the model's built in loss.

In [ ]:
#|exporti
warnings.simplefilter("ignore")
hf_logging.set_verbosity_error()

In [ ]:
#|export
class HuggingFaceLoss(BaseLoss):
    "To use the Hugging Face model's built in loss function, pass this loss to `Learner`"
    def __init__(self, **kwargs):
        self.func = None

    def __call__(self, inp:Tensor, targ:Tensor|None=None, **kwargs):
        return tensor(0.0)

In [ ]:
#|export
class HuggingFaceWrapper(nn.Module, ModuleUtilsMixin):
    "A minimal compatibility wrapper between a Hugging Face model and `Learner`"
    def __init__(
        self,
        model: PreTrainedModel, # Hugging Face compatible model
    ):
        super().__init__()
        self.hf_model = model
        self._forward_args = inspect.getfullargspec(self.hf_model.forward).args

    def forward(self, x:Dict):
        return self.hf_model(**{k:v for k,v in x.items() if k in self._forward_args})

In practice, you won't need to use the <code>HuggingFaceWrapper</code> as `HuggingFaceCallback` will automatically add it for you.

In [ ]:
#|export
class HuggingFaceCallback(Callback):
    "Provides compatibility between fastai's `Learner`, the Transformers model, & `HuggingFaceLoader`"
    run_valid = True
    def __init__(self,
        labels:str|None='labels', # Input batch labels key. Set to None if input doesn't contain labels
        loss:str='loss', # Model output loss key
        logits:str='logits', # Model output logits key
        unwrap:bool=False, # After training completes, unwrap the Transformers model
    ):
        self._label_key, self._loss_key = labels, loss
        self._logit_key, self.unwrap = logits, unwrap

    def after_create(self):
        self._model_loss = isinstance(self.learn.loss_func, HuggingFaceLoss)
        if not isinstance(self.model, HuggingFaceWrapper) and not isinstance(self.model, dynamo.OptimizedModule):
            self.learn.model = HuggingFaceWrapper(self.learn.model)
            self.learn.hf_model = self.learn.model.hf_model

    def before_batch(self):
        self._loss = None
        if self._label_key is not None:
            if not self._model_loss:
                self.learn.yb = (self.xb[0].pop(self._label_key),)
            else:
                self.learn.yb = (self.xb[0][self._label_key],)
        else:
            self.learn.yb = (1,)

    def after_pred(self):
        outputs = self.learn.pred
        if self._model_loss:
            self._loss = to_float(outputs[self._loss_key])
        self.learn.pred = outputs.get(self._logit_key, None)

    def after_loss(self):
        if self._model_loss:
            self.learn.loss_grad = self._loss
            self.learn.loss = self.learn.loss_grad.clone()
        else:
            self.xb[0][self._label_key] = self.learn.yb[0]

    def after_fit(self):
        if self.unwrap:
            if isinstance(self.learn.model, dynamo.OptimizedModule) and hasattr(self.learn, 'compiler'):
                self.learn.compiler._reset_compiled()
            if isinstance(self.model, HuggingFaceWrapper):
                self.learn.model = self.learn.model.hf_model

<code>HuggingFaceCallback</code> automatically wraps a Transformer model with the `HuggingFaceWrapper` for compatibility with `fastai.learner.Learner`. The original Transformers model is accessable via `Learner.hf_model`.

If `HuggingFaceLoss` is passed to `Learner`, then <code>HuggingFaceCallback</code> will use the Hugging Face model's built in loss.

If any other loss function is passed to `Learner`, <code>HuggingFaceCallback</code> will prevent the built-in loss from being calculated and will use the `Learner` loss function instead.

If `labels=None`, then <code>HuggingFaceCallback</code> will not attempt to assign a fastai target from the Hugging Face input batch. The default fastai and fastxtend metrics will not work without labels.

After training, the <code>HuggingFaceCallback</code> can automatically unwrap model if `unwrap=True`.

In [ ]:
#|export
class HuggingFaceLoader(_DataLoader):
    "A minimal compatibility DataLoader between a Hugging Face and `Learner`"
    def __init__(self,
        dataset:Dataset, # dataset from which to load the data
        batch_size:int, # Batch size
        shuffle:bool|None = None, # Randomize the order of data at each epoch (default: False)
        sampler:Sampler|Iterable|None = None, # Determines how to draw samples from the dataset. Cannot be used with shuffle.
        batch_sampler:Sampler[Sequence]|Iterable[Sequence]|None = None, # Rreturns a batch of indices at a time. Cannot be used with batch_size, shuffle, sampler, or drop_last.
        num_workers:int=0, # Number of processes to use for data loading. 0 means using the main process (default: 0).
        collate_fn:_collate_fn_t|None = None, # Function that merges a list of samples into a mini-batch of Tensors. Used for map-style datasets.
        pin_memory:bool=False, # Copy Tensors into device/CUDA pinned memory before returning them
        drop_last:bool=False, # Drop the last incomplete batch if the dataset size is not divisible by the batch size
        timeout:float=0, # Timeout value for collecting a batch from workers
        worker_init_fn:_worker_init_fn_t|None = None, # called on each worker subprocess with the worker id as input
        multiprocessing_context=None,
        generator=None,
        prefetch_factor:int|None=None, # number of batches loaded in advance by each worker
        persistent_workers:bool=False, # if True, the data loader will not shutdown the worker processes after a dataset has been consumed once
        pin_memory_device:str= "", # the data loader will copy Tensors into device pinned memory before returning them if pin_memory is set to true
    ):
        super().__init__(
            dataset=dataset, batch_size=batch_size, shuffle=shuffle, sampler=sampler,
            batch_sampler=batch_sampler, num_workers=num_workers, collate_fn=collate_fn,
            pin_memory=pin_memory, drop_last=drop_last, timeout=timeout,
            worker_init_fn=worker_init_fn, multiprocessing_context=multiprocessing_context,
            generator=generator, prefetch_factor=prefetch_factor, persistent_workers=persistent_workers,
            pin_memory_device=pin_memory_device
        )

    @property
    def bs(self) -> int:
        "Number of items a batch"
        return self.batch_size

    def __iter__(self):
        for b in super().__iter__():
            yield (b,)

Hugging Face Datasets, and thus DataLoaders, return dictionary objects while the `fastai.learner.Learner` expects tuples. <code>HuggingFaceLoader</code> is a PyTorch DataLoader which wraps the Hugging Face batch dictionary in a tuple for `Learner` compatibility. It is otherwise identical to a PyTorch DataLoader.

## Example

In [ ]:
#|hide
#|cuda
import os
import tempfile

from datasets import concatenate_datasets, load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers.utils import logging as hf_logging

from fastcore.basics import num_cpus

from fastai.data.core import DataLoaders
from fastai.learner import Learner

from fastxtend.callback import compiler
from fastxtend.callback import amp
from fastxtend.metrics import Accuracy
from fastxtend.optimizer.stableadam import stableadam
from fastxtend import schedulers
from fastxtend.utils import less_random

from fastxtend.imports import *
from fastxtend.utils import *

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# silence hugging face warnings
warnings.simplefilter("ignore")
hf_logging.set_verbosity_error()

In this example, we'll use Hugging Face [Transformers](https://huggingface.co/docs/transformers) along with fastai & fastxtend to train a [DistilRoBERTa](https://huggingface.co/distilroberta-base) on a [IMDb](https://huggingface.co/datasets/imdb) subset.

:::{.callout-note collapse="true"}
#### Note: Transformers Documentation

This example is based on the Transformers documentation [sequence classification example](https://huggingface.co/docs/transformers/tasks/sequence_classification).
:::

### Setup Transformer Objects

First, we'll grab the DistilRoBERTa tokenizer and model from the Transformers Auto methods.

In [ ]:
#|cuda
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=2)

Next, download IMDb using [Dataset's](https://huggingface.co/docs/datasets) `load_dataset`. In this example, we'll use a subset of IMDb.

In [ ]:
#|cuda
imdb = load_dataset('imdb')
with less_random():
    imdb['train'] = imdb['train'].shuffle().select(range(5000))
    imdb['test'] = imdb['test'].shuffle().select(range(1000))

Next, we'll tokenize the data using Dataset's `map` method.

In [ ]:
#|cuda
def tokenize_data(batch, tokenizer):
    return tokenizer(batch['text'], truncation=True)

imdb['train'] = imdb['train'].map(
    partial(tokenize_data, tokenizer=tokenizer),
    remove_columns='text', batched=True, batch_size=512, num_proc=num_cpus(),
)

imdb['test'] = imdb['test'].map(
    partial(tokenize_data, tokenizer=tokenizer),
    remove_columns='text', batched=True, batch_size=512, num_proc=num_cpus(),
)

### Define the DataLoader

We need to use fastxtend's `HuggingFaceLoader` instead of the PyTorch `DataLoader`. <code>HuggingFaceLoader</code> is a simple wrapper around a PyTorch `DataLoader` which returns Transformer's dictionary batches in tuples as the `fastai.learner.Learner` expects. It is otherwise identical to the PyTorch DataLoader.

After creating the train and valid <code>HuggingFaceLoader</code>, we need to wrap them in `fastai.data.core.DataLoaders`.

In [ ]:
#|cuda
with less_random():
    train_dataloader = HuggingFaceLoader(
        imdb['train'].with_format('torch'), batch_size=16,
        collate_fn=DataCollatorWithPadding(tokenizer), shuffle=True,
        drop_last=True, num_workers=num_cpus()
    )

    valid_dataloader = HuggingFaceLoader(
        imdb['test'].with_format('torch'), batch_size=16,
        collate_fn=DataCollatorWithPadding(tokenizer), shuffle=False,
        drop_last=False, num_workers=num_cpus()
    )

    dls = DataLoaders(train_dataloader, valid_dataloader)

### Create a Learner and Train

Finally, we'll create the `Learner` to train DistilRoBERTa on IMDb. We'll pass in the `HuggingFaceCallback` to `cbs` to handle loss function compatibility between Transformers and fastai.

Transformer models contain an internal loss method, which we'll use by passing `HuggingFaceLoss` to `loss_func`.

:::{.callout-note collapse="true"}
#### Note: Non-Default Transformers Keys

<code>HuggingFaceCallback</code> expects the Transformer model to have and output logits and loss keys. If these exist but are named differently, you'll need to pass the non-standard key names to <code>HuggingFaceCallback</code>.

If your input doesn't have a label key, perhaps because you are pretraining a causal language model, you should set it to `None`.
:::

We now can use any fastai and/or fastxtend callbacks, optimizers, or metrics to train our Transformers model like usual.

In [ ]:
#|cuda
with less_random():
    learn = Learner(dls, model, loss_func=HuggingFaceLoss(),
                    opt_func=stableadam(foreach=True),
                    metrics=Accuracy(), cbs=HuggingFaceCallback).to_bf16()

    learn.fit_flat_warmup(3, lr=8e-4, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.691708,0.690203,0.492000,00:38
1,0.510412,0.409681,0.854000,00:37
2,0.282954,0.300484,0.873000,00:38


In [ ]:
#|hide
#|cuda
model = None
free_gpu_memory(learn, dls)

If we want to use our own loss, such as `nn.CrossEntropyLoss` with label smoothing, we could pass in any PyTorch compatible loss function to `Learner` and <code>HuggingFaceCallback</code> will automatically use it instead of DistilRoBERTa's internal loss function.

In this example, we use fastxtend's `CompilerCallback` via the `Learner.compile` convenience method to accelerate training throughput using `torch.compile`. After compiling the model in the first epoch, training speed is increased, and memory usage is reduced. In this small example it's an overall loss, but we'd want to compile DistilRoBERTa if training on the entirety of IMDb.

:::{.callout-warning collapse="true"}
#### Warning: Dynamic Requires PyTorch 2.1+

Compiling the model with `compile(dynamic=True)` requires a minimum of Pytorch 2.1. Dynamic shapes does not work in PyTorch 2.0.
:::

In [ ]:
#|cuda
model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=2)

with less_random():
    learn = Learner(dls, model, loss_func=nn.CrossEntropyLoss(label_smoothing=0.1),
                    opt_func=stableadam(foreach=True), metrics=Accuracy(),
                    cbs=HuggingFaceCallback).to_bf16().compile(dynamic=True)

    learn.fit_flat_warmup(3, lr=8e-4, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.686346,0.677865,0.658000,01:25
1,0.423131,0.383354,0.886000,00:27
2,0.355547,0.374400,0.887000,00:27


### Accessing and Saving the Model

The the original Transformers model is accessable via `Learner.model.hf_model` or `Learner.hf_model` (both point to the same object).

We can use any Transformers method to save the model, such as `save_pretrained`.

In [ ]:
#|hide
#|cuda
temp_path = tempfile.TemporaryDirectory(dir=learn.path)
model_path = temp_path.name

In [ ]:
#|cuda
learn.hf_model.save_pretrained(model_path)

In [ ]:
#|hide
#|cuda
temp_path.cleanup()